# Download Data

In [3]:
!pip install yfinance
#https://pypi.org/project/yfinance/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [1]:
import yfinance as yf
import datetime as dt

start = dt.datetime(2020,1,1)
end = dt.datetime.now()
btc = yf.Ticker('BTC-USD')
eth = yf.Ticker('ETH-USD')
gold = yf.Ticker('GC=F')

In [2]:
btc_history = btc.history(period="max")
eth_history = eth.history(period="max")
gold_history = gold.history(period='max')

In [11]:
#find the latest start date from the 3 data sets so we can merge into a single complete dataframe
start_dates = [btc_history.index[0], eth_history.index[0], gold_history.index[0]]
max_date = max(start_dates)

data = yf.download("BTC-USD ETH-USD GC=F", start=max_date)

start_dates, max_date

[*********************100%***********************]  3 of 3 completed


([Timestamp('2014-09-17 00:00:00'),
  Timestamp('2017-11-09 00:00:00'),
  Timestamp('2000-08-30 00:00:00')],
 Timestamp('2017-11-09 00:00:00'))

In [12]:
data.head()

Adj Close                                 Close              \
                BTC-USD     ETH-USD         GC=F      BTC-USD     ETH-USD   
Date                                                                        
2017-11-09  7143.580078  320.884003  1285.599976  7143.580078  320.884003   
2017-11-10  6618.140137  299.252991  1272.400024  6618.140137  299.252991   
2017-11-11  6357.600098  314.681000          NaN  6357.600098  314.681000   
2017-11-12  5950.069824  307.907990          NaN  5950.069824  307.907990   
2017-11-13  6559.490234  316.716003  1277.300049  6559.490234  316.716003   

                                High                                   Low  \
                   GC=F      BTC-USD     ETH-USD         GC=F      BTC-USD   
Date                                                                         
2017-11-09  1285.599976  7446.830078  329.451996  1286.900024  7101.520020   
2017-11-10  1272.400024  7312.000000  324.717987  1283.500000  6436.870117   
2017-11-11          NaN  6873.149902  319.453003          NaN  6204.220215   
2017-11-12          NaN  6625.049805  319.153015          NaN  5519.009766   
2017-11-13  1277.300049  6811.189941  328.415009  1277.300049  5844.290039   

                                            Open                           \
               ETH-USD         GC=F      BTC-USD     ETH-USD         GC=F   
Date                                                                        
2017-11-09  307.056000  1279.699951  7446.830078  308.644989  1279.699951   
2017-11-10  294.541992  1272.400024  7173.729980  320.670990  1283.500000   
2017-11-11  298.191986          NaN  6618.609863  298.585999          NaN   
2017-11-12  298.513000          NaN  6295.450195  314.690002          NaN   
2017-11-13  307.024994  1277.000000  5938.250000  307.024994  1277.300049   

                Volume                    
               BTC-USD     ETH-USD  GC=F  
Date                                      
2017-11-09  3226249984   893249984  18.0  
2017-11-10  5208249856   885985984  10.0  
2017-11-11  4908680192   842300992   NaN  
2017-11-12  8957349888  1613479936   NaN  
2017-11-13  6263249920  1041889984   3.0

In [13]:
raw_data = data.copy()

#Gold trading is closed on weekends (Crypto is 24/7), so we need to impute some missing values
#use 'ffill', meaing Friday's close will forward fill Saturday & Sunday
raw_data.fillna(method='ffill', inplace=True)

#collapse the 2 row header into a single row
raw_data.columns = raw_data.columns.map('-'.join)

#rename the columns for easy of access
raw_data.columns = ['BTC-adj_close', 'ETH-adj_close', 'AU-adj_close', 
                'BTC-close', 'ETH-close', 'AU-close', 
                'BTC-high', 'ETH-high', 'AU-high',
                'BTC-low', 'ETH-low', 'AU-low',
                'BTC-open', 'ETH-open', 'AU-open',
                'BTC-volume', 'ETH-volume', 'AU-volume',
                ]

#reorder for readability
raw_data = raw_data[['BTC-open','BTC-low','BTC-high','BTC-close','BTC-adj_close','BTC-volume',
                     'ETH-open','ETH-low','ETH-high','ETH-close','ETH-adj_close','ETH-volume',
                     'AU-open','AU-low','AU-high','AU-close','AU-adj_close','AU-volume']]

raw_data.head()

,BTC-open,BTC-low,BTC-high,BTC-close,BTC-adj_close,BTC-volume,ETH-open,ETH-low,ETH-high,ETH-close,ETH-adj_close,ETH-volume,AU-open,AU-low,AU-high,AU-close,AU-adj_close,AU-volume
Date,,,,,,,,,,,,,,,,,,
2017-11-09,7446.830078,7101.520020,7446.830078,7143.580078,7143.580078,3226249984,308.644989,307.056000,329.451996,320.884003,320.884003,893249984,1279.699951,1279.699951,1286.900024,1285.599976,1285.599976,18.0
2017-11-10,7173.729980,6436.870117,7312.000000,6618.140137,6618.140137,5208249856,320.670990,294.541992,324.717987,299.252991,299.252991,885985984,1283.500000,1272.400024,1283.500000,1272.400024,1272.400024,10.0
2017-11-11,6618.609863,6204.220215,6873.149902,6357.600098,6357.600098,4908680192,298.585999,298.191986,319.453003,314.681000,314.681000,842300992,1283.500000,1272.400024,1283.500000,1272.400024,1272.400024,10.0
2017-11-12,6295.450195,5519.009766,6625.049805,5950.069824,5950.069824,8957349888,314.690002,298.513000,319.153015,307.907990,307.907990,1613479936,1283.500000,1272.400024,1283.500000,1272.400024,1272.400024,10.0
2017-11-13,5938.250000,5844.290039,6811.189941,6559.490234,6559.490234,6263249920,307.024994,307.024994,328.415009,316.716003,316.716003,1041889984,1277.300049,1277.000000,1277.300049,1277.300049,1277.300049,3.0


In [10]:
#save the raw data file to CSV
raw_data.to_csv('raw_data.csv')